# Mercator fix test

In this notebook we will try to implement the addition of extra dummy points to the .json that goes into the mapping tool to correct the distortion created by the Mercator projection in the grid data.

In [2]:
import json
import pickle
import numpy as np
from pathlib import Path

In [8]:
def array_to_coords(array, round_to=3, lon_range=(-180, 180), lat_range=(-60, 80), threshold=0, normalize_by=1.):
    norm = array/normalize_by
    lats = np.arange(lat_range[1], lat_range[0], -(lat_range[1] - lat_range[0])/array.shape[0])
    lons = np.arange(lon_range[0], lon_range[1],  (lon_range[1] - lon_range[0])/array.shape[1])
    rows = list()
    rows = [[round(float(lats[i]), round_to), 
             round(float(lons[j]), round_to), 
             round(float(norm[i, j]), round_to)]
            for j in range(array.shape[1]) 
            for i in range(array.shape[0])
            if array[i, j] > threshold]  # Do not store values below or equal to the threshold (e.g. zeroes)
    return rows

In [16]:
def array_to_json_extra(array, destination, value_name="value", value_type="real", 
        round_to=3, lon_range=(-180, 180), lat_range=(-60, 80), threshold=0, normalize_by=1.,
        lat_extra_1=55, lat_extra_2=70,
    ):
    data = {}
    fields = [
        {"name": "latitude", "format": "", "type": "real"},
        {"name": "longitude", "format": "", "type": "real"},
        {"name": value_name, "format": "", "type": value_type},
    ]
    data["fields"] = fields
    rows = array_to_coords(array, 
        round_to=round_to, 
        lon_range=lon_range, 
        lat_range=lat_range, 
        threshold=threshold,
        normalize_by=normalize_by,
    )
    
    lat_space = (lat_range[1] - lat_range[0]) / array.shape[0]
    extra_rows_1 = [[round(float(row[0] - lat_space/2), round_to), row[1], row[2]]
                  for row in rows if (abs(row[0]) >= lat_extra_1) and (abs(row[0]) < lat_extra_2)]
    
    extra_rows_2 = [[round(float(row[0] - lat_space/3), round_to), row[1], row[2]]
                  for row in rows if (abs(row[0]) >= lat_extra_2)]
    extra_rows_2 += [[round(float(row[0] - lat_space*2/3), round_to), row[1], row[2]]
                  for row in rows if (abs(row[0]) >= lat_extra_2)]
    
    data["rows"] = rows + extra_rows_1 + extra_rows_2
    with open(destination, 'w') as file:
        json.dump(data, file)

In [17]:
data_pr = Path("../data/processed/")

In [20]:
def apply_fix(dataset):
    with open(data_pr/f"{dataset}/{dataset}.pickle", "rb") as f:
        array = pickle.load(f)
    array_to_json_extra(array, data_pr/f"{dataset}/{dataset}_mercfix.json")

In [22]:
datasets = ["mammals", "cultivated", "urban", "floods", "deforestation", "pastures", "treecover", "population"]
for dataset in datasets:
    apply_fix(dataset)